In [80]:


import clingo
import clingo.ast

In [81]:
program = """

%domA.

domA(0).

domA(1) :- domA(A).

"""

In [82]:
class NonGroundReifyTransformer(clingo.ast.Transformer):
    pos = clingo.ast.Position('<string>', 1, 1)
    loc = clingo.ast.Location(pos, pos)

    def _ast_sym(self, sym: clingo.Symbol):
        return clingo.ast.SymbolicAtom(clingo.ast.SymbolicTerm(self.loc, sym))

    def _ast_str(self, string: str):
        return self._ast_sym(clingo.String(string))

    def _ast_num(self, num: int):
        return self._ast_sym(clingo.Number(num))

    def visit_SymbolicTerm(self, term: clingo.ast.AST):
        print("Term:", term)
        symbol = term.symbol
        symbol_node = clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'symbol',
                                                                  [self._ast_sym(symbol)],
                                                                  False))
        print("Meta-Term:", symbol_node)
        return symbol_node

    def visit_Variable(self, variable: clingo.ast.AST):
        print("Variable:", variable)
        name = variable.name
        variable_node = clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'variable', [
            clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'name', [
                self._ast_str(name)
            ], False))], False))
        print("Meta-Variable:", variable_node)
        return variable_node

    def visit_Function(self, function: clingo.ast.AST):
        print("Function:", function)
        name = function.name
        arguments = function.arguments
        external = function.external
        function_node = clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'function', [
            clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'name', [self._ast_str(name)], False)),
            clingo.ast.SymbolicAtom(
                clingo.ast.Function(self.loc, 'arguments', [self.visit(argument) for argument in arguments], False)),
            clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'external', [self._ast_num(external)], False))
        ], False))
        print("Meta-Function:", function_node)
        return function_node

    def visit_Literal(self, literal: clingo.ast.AST):
        print("Literal:", literal)
        sign = literal.sign
        atom = literal.atom
        literal_node = clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'literal', [
            clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'sign', [self._ast_num(sign)], False)),
            clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'atom', [
                clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'symbol', [
                    self.visit(atom)
                ], False)),
            ], False))
        ], False))
        print("Meta-Literal:", literal_node)
        return literal_node

    def visit_Rule(self, rule: clingo.ast.AST):
        print("Rule:", rule)
        meta_rule = self.visit_children(rule)
        head = clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'head', [meta_rule['head']], False))
        body = clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'body', [clingo.ast.Function(self.loc, 'elements', meta_rule.get('body', ()), False)], False))
        rule_head = clingo.ast.Literal(self.loc, clingo.ast.Sign.NoSign, clingo.ast.SymbolicAtom(
            clingo.ast.Function(self.loc, 'rule', (head,
                                                   body,
                                                   clingo.ast.SymbolicAtom(clingo.ast.Function(self.loc, 'external', [self._ast_num(0)], False))),
                                False)))
        rule_node = clingo.ast.Rule(self.loc, rule_head, ())
        print("Meta-Rule:", rule_node)
        return rule_node



In [83]:
ctl = clingo.Control()
ctl.configuration.solve.models = 0


In [84]:
ngrt = NonGroundReifyTransformer()

nodes = []
clingo.ast.parse_string(program, lambda stm: nodes.append(ngrt.visit(stm)))
print("-"*80)
with clingo.ast.ProgramBuilder(ctl) as bld:
    for node in nodes:
        print(node.ast_type)
        bld.add(node)
        print(node)


Rule: domA(0).
Literal: domA(0)
Function: domA(0)
Term: 0
Meta-Term: symbol(0)
Meta-Function: function(name("domA"),arguments(symbol(0)),external(0))
Meta-Literal: literal(sign(0),atom(symbol(function(name("domA"),arguments(symbol(0)),external(0)))))
Meta-Rule: rule(head(literal(sign(0),atom(symbol(function(name("domA"),arguments(symbol(0)),external(0)))))),body(elements),external(0)).
Rule: domA(1) :- domA(A).
Literal: domA(1)
Function: domA(1)
Term: 1
Meta-Term: symbol(1)
Meta-Function: function(name("domA"),arguments(symbol(1)),external(0))
Meta-Literal: literal(sign(0),atom(symbol(function(name("domA"),arguments(symbol(1)),external(0)))))
Literal: domA(A)
Function: domA(A)
Variable: A
Meta-Variable: variable(name("A"))
Meta-Function: function(name("domA"),arguments(variable(name("A"))),external(0))
Meta-Literal: literal(sign(0),atom(symbol(function(name("domA"),arguments(variable(name("A"))),external(0)))))
Meta-Rule: rule(head(literal(sign(0),atom(symbol(function(name("domA"),argu

RuntimeError: invalid ast: term expected

In [ ]:
ctl.ground([('base', [])])

In [ ]:
with ctl.solve(yield_=True) as solve_handle:
    models = []
    for model in solve_handle:
        symbols = model.symbols(atoms=True)
        models.append(symbols)

In [ ]:
print(' '.join(map(str, models[0])))